# HW №2.1

**Промежуточная задача** - преобразовать данные в `pandas.DataFrame` вида `{user, item, order}`, где `order` - порядковый номер с конца (0 - самый "свежий" лайк, чем больше `order`, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования




*   Между `train` и `test` не должно быть общих пользователей
*   Количество фолдов задается через параметр класса `n_folds`
*   В `test` должно быть не более `p` последних треков (параметр класса `p`)

## 0. Предподготовка 

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

In [11]:
PATH = Path('/home/iuliiasolomennikova/Desktop/RecoService/data')

In [ ]:
!unzip '{PATH}/dataframe.csv.zip'

In [14]:
df = pd.read_csv(PATH/'dataframe.csv')

## 1. Промежуточная задача

In [15]:
# предотвращаем пересечение id пользователей в трейне и тесте выборке
max_id_train = df[df.is_train == 1].user_id.max()
df.loc[df.is_train == 0, "user_id"] += max_id_train

# оставляем только нужные столбцы и rename
df = df[['user_id', 'track_id']]
df = df.rename(columns={
    "user_id": "user", 
    "track_id": "item",
    })


In [16]:
# добавляем столбец с порядком "свежести" лайков
df['order'] = df.groupby('user').cumcount() 

df

,user,item,order
0,0,333396,0
1,0,267089,1
2,0,155959,2
3,0,353335,3
4,0,414000,4
...,...,...,...
117450829,1449996,448288,34
117450830,1449996,1343,35
117450831,1449996,86420,36
117450832,1449996,186436,37


## 2. Итоговая задача

In [17]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df):
        users = df.user.unique()
        num_users = len(users)

        np.random.seed(self.random_seed)
        np.random.shuffle(users)

        fold_sizes = np.full(self.n_folds, num_users // self.n_folds, dtype=int)
        fold_sizes[: num_users % self.n_folds] += 1
        current = 0

        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            # Для тестовой части берутся первые p взаимодействий пользователей, 
            # попавших в тестовую часть фолда
            test_mask = (df.user.isin(test_fold_users)) & (df.order < self.p)
            # Для тренировочной части берутся все взаимодействия пользователей, 
            # не попавших в тестовую часть фолда
            train_mask = ~df.user.isin(test_fold_users)
            
            yield train_mask, test_mask

In [18]:
# используем наш класс
cv = UsersKFoldPOut(n_folds=6, p=1)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
    train = df[train_mask]
    test = df[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 97913558, Test: 241667
Fold#1 | Train: 97913617, Test: 241666
Fold#2 | Train: 97913617, Test: 241666
Fold#3 | Train: 97913617, Test: 241666
Fold#4 | Train: 97913617, Test: 241666
Fold#5 | Train: 97913617, Test: 241666
